In [1]:
import torch
torch.cuda.empty_cache()
if torch.cuda.is_available():
    print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Using CPU instead.")

CUDA is available. Using GPU: NVIDIA L40S


In [ ]:
from flask import Flask, request, jsonify
from flask import make_response
from flask_cors import CORS
import traceback
import requests 
from process import analyze_with_llama, transcribe_audio_with_whisper, convert_audio_to_wav, process_text
from datetime import datetime  
import json


app = Flask(__name__)
CORS(app, supports_credentials=True)


@app.route('/analyze', methods=['GET'])
def analyze():
    try:
        file_url = request.args.get('fileUrl')
        if not file_url:
            return jsonify({"error": "No file URL provided"}), 400
        print(f"Received file URL: {file_url}")

        audio_file = "temp_audio_file.m4a"
        with open(audio_file, 'wb') as f:
            f.write(requests.get(file_url).content)

        wav_file = "temp.wav"
        convert_audio_to_wav(audio_file, wav_file)

        transcribed_text = transcribe_audio_with_whisper(wav_file)
        print("Transcribed text:", transcribed_text)
        processed_text = process_text(transcribed_text)

        analysis_result = analyze_with_llama(processed_text)
        result = {"input": processed_text, "output": analysis_result}

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"results/analysis_result_{timestamp}.json"
        with open(output_file, 'w') as json_file:
            json.dump(result, json_file, indent=4)
        
        print(f"Analysis result saved to {output_file}")
        return jsonify(result), 200

    except Exception as e: 
        print("Exception occurred: ", str(e)) 
        traceback.print_exc() 
        return jsonify({"error": str(e)}), 500 

if __name__ == '__main__': 
    app.run(host='0.0.0.0', port=5000)

100%|█████████████████████████████████████| 72.1M/72.1M [00:22<00:00, 3.34MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.27.0.2:5000
Press CTRL+C to quit
100.64.0.33 - - [03/Nov/2024 10:33:52] "GET / HTTP/1.1" 404 -
100.64.0.30 - - [03/Nov/2024 10:33:55] "GET / HTTP/1.1" 404 -
100.64.0.21 - - [03/Nov/2024 10:34:12] "OPTIONS /analyze?fileUrl=https://firebasestorage.googleapis.com/v0/b/data5703-6c8d4.appspot.com/o/uploads%252F2409_servicesAustralia_linkdin_media_blind_screenreader_voiceOver_Pat_patsAudio.m4a?alt%3Dmedia%26token%3D2f24e722-4a08-427d-8a77-5b3b0e546e1e HTTP/1.1" 200 -


Received file URL: https://firebasestorage.googleapis.com/v0/b/data5703-6c8d4.appspot.com/o/uploads%2F2409_servicesAustralia_linkdin_media_blind_screenreader_voiceOver_Pat_patsAudio.m4a?alt=media&token=2f24e722-4a08-427d-8a77-5b3b0e546e1e


...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processing line: I can access the information sometimes but it reads out hundreds of things that are not even there and it does not read out anything correctly a lot of the time 


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processing line: I am going to slow that down right down and just this is what 


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
